In [11]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
import csv

#Sqlalchemy imports
from sqlalchemy import Column, Float, Integer, String,DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect
import pymysql
pymysql.install_as_MySQLdb()

In [12]:
#Clean data CSV import
hmeasures = "raw_data/clean_hawaii_measurements.csv"
hstations = "raw_data/clean_hawaii_stations.csv"
hmeasures_df = pd.read_csv(hmeasures)
hstations_df = pd.read_csv(hstations)

#Data structure overview
print("hstations_df column information: \n")
print(hstations_df.columns,hstations_df.dtypes) 
print("=============================")
print("hmeasures_df column information: \n")
print(hmeasures_df.columns,hmeasures_df.dtypes)

hstations_df column information: 

Index(['station', 'name', 'latitude', 'longitude', 'elevation'], dtype='object') station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object
hmeasures_df column information: 

Index(['station', 'date', 'prcp', 'tobs'], dtype='object') station     object
date        object
prcp       float64
tobs         int64
dtype: object


In [13]:
Base = declarative_base()

class Stations(Base):
    __tablename__ = "Stations"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

class Measurements(Base):
    __tablename__ = "Measurements"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(DateTime)
    prcp = Column(Float)
    tobs = Column(Float)
    

In [15]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

session = Session(bind=engine)
                    
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html                   
hmeasures_df.to_sql('measurements',engine, if_exists='replace',index=True)
hstations_df.to_sql('stations',engine, if_exists='replace',index=True)

#csv ---> dataframe ---> to_dict(orient='records') --> 
#def populate(engine, table, csvfile):
    #conn = engine.connect()
    #conn.execute(table.insert(), data)

C:\Users\tyu-dev\Anaconda3\envs\pythondata\lib\site-packages\pandas\io\sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


OperationalError: (sqlite3.OperationalError) near "PRIMARY": syntax error [SQL: 'ALTER TABLE `measures` ADD PRIMARY KEY (`id`);']

In [ ]:
#Confirm table and table data import in hawaii.sqlite
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
Base.metadata.create_all(engine)
session.commit()

In [ ]:
data = engine.execute("PRAGMA table_info([stations]);")
for item in data:
    print(item)

In [ ]:
data = engine.execute("PRAGMA table_info([measurements]);")
for item in data:
    print(item)

In [ ]:
data = engine.execute("SELECT * FROM stations")
for item in data:
    print(item)

In [ ]:
data = engine.execute("SELECT * FROM measurements")
for item in data:
    print(item)